In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as model
import regex as re
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import stats
from datetime import date
import geo_tools as gt

In [3]:
def process_data(GSE_list, info= "age"):
    
    """Takes in a list of GSEs, returns a corresponding X_df and y."""
    
    # making X
    GSE_df_dict = {}
    GSE_df_list = []

    for GSE in GSE_list:

        GSE_df = gt.series(GSE).transpose()

    #     let's make it multiindex with the GSEs
        multi_index_arrays = [[], []]
        for GSM in GSE_df.index:
            multi_index_arrays[0].append(GSE)
            multi_index_arrays[1].append(GSM)

        GSE_df.index =  pd.MultiIndex.from_arrays(multi_index_arrays)

        # we need to deal with the missing values
        GSE_df.fillna(inplace= True, method= 'ffill')
    #     print (GSE_df.equals(GSE_df.dropna(axis= 'columns')))

        GSE_df_dict[GSE] = GSE_df
        GSE_df_list.append(GSE_df)

    X_df = pd.concat(GSE_df_list)

    # now let's manually remove the rows that still contain invalid values that the pandas library wasn't able to deal with in the
    # above cells

    samples_na = X_df.isna().any(axis= 1)

    for sample in samples_na.index:
        if samples_na.loc[sample]:
            X_df.drop(labels= sample, inplace= True)

    y = []
    for row in X_df.index:
        y.append(gt.info(row[0], row[1], info= info))
        
    index_arr = [re.match("cg", label) == None for label in X_df.columns]
    bad_labels = np.array(X_df.columns)[index_arr]
    
    X_df.drop(labels= bad_labels, axis= "columns", inplace= True)
    
    # add stuff to drop rows with more than 10 missing values
        
    return X_df, y

In [4]:
temp_data_28k = ["GSE41037"]

In [5]:
temp_data_450k = ["GSE41169"]

In [6]:
temp_X_28k_wb, _ = process_data(temp_data_28k)

In [7]:
temp_X_450k_wb, _ = process_data(temp_data_450k)

In [8]:
# 450k array doesn't have some of the 28k array columns
shared_CpGs = list(set(temp_X_28k_wb.columns).intersection(set(temp_X_450k_wb.columns)))

In [9]:
del temp_data_28k, temp_data_450k, temp_X_28k_wb, temp_X_450k_wb, _

## Figure 4A

In [10]:
hannum_data = ["GSE40279"]

In [11]:
hannum_X, hannum_y = process_data(hannum_data)

In [12]:
hannum_CpGs = hannum_X.columns

In [13]:
scaler = StandardScaler()

In [14]:
hannum_X = scaler.fit_transform(hannum_X, hannum_y)

In [ ]:
hannum_X_train, hannum_X_test, hannum_y_train, hannum_y_test = train_test_split(hannum_X, hannum_y, test_size= 0.20)

hannum_data_model = model.ElasticNetCV(alphas= [0.02, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1], l1_ratio= [0.1, 0.3, 0.5, 0.8, 0.95], selection= "random")

In [ ]:
han_alpha = hannum_data_model.alpha
han_l1_ratio = hannum_data_model.l1_ratio

In [ ]:
hannum_data_model.fit(hannum_X_train, hannum_y_train)

In [ ]:
age_pred = hannum_data_model.predict(hannum_X_test)
plt.scatter(hannum_y_test, age_pred, label= "samples", marker= "+")
plt.title("DNAm pred. vs actual age, Hannum\nr= " + str(stats.pearsonr(hannum_y_test, age_pred)[0]), loc= "right")
plt.xlim(0, 100)
plt.ylim(0, 100)
accuracy_line = [0, 100]
plt.plot(accuracy_line, accuracy_line)
plt.xlabel("actual age")
plt.ylabel("DNAm predicted age")
plt.legend()
plt.savefig(fname= "hannum_data_age_model")
plt.show()

In [ ]:
del hannum_X_train, hannum_X_test, hannum_y_train, hannum_y_test

now let's train a model where we randomize ages in the middle but keep young and old cohorts' actual age

In [ ]:
hannum_y_cohorts = hannum_y.copy()

In [ ]:
rand_age_ind = []
norm_age_ind = []
for i in range(len(hannum_y_cohorts)):
    age = hannum_y_cohorts[i]
    if age >= 43 and age < 80:
        hannum_y_cohorts[i] = np.random.randint(43, 80)
        rand_age_ind.append(True)
        norm_age_ind.append(False)
    else:
        rand_age_ind.append(False)
        norm_age_ind.append(True)

In [ ]:
hannum_X_train, hannum_X_test, hannum_y_train, hannum_y_test, rand_age_ind_train, rand_age_ind_test, norm_age_ind_train, \
norm_age_ind_test = train_test_split(hannum_X, hannum_y, rand_age_ind, norm_age_ind, test_size= 0.20)

In [ ]:
hannum_data_model_rand = model.ElasticNet(alpha= han_alpha, l1_ratio= han_l1_ratio)

In [ ]:
hannum_data_model_rand.fit(hannum_X_train, hannum_y_train)

In [ ]:
han_coh_pred = hannum_data_model_coh.predict(hannum_X_test)

In [ ]:
plt.scatter(np.array(hannum_y_test)[norm_age_ind_test], np.array(han_coh_pred)[norm_age_ind_test], label= "normal age", marker= "+")
plt.scatter(np.array(hannum_y_test)[rand_age_ind_test], np.array(han_coh_pred)[rand_age_ind_test], label= "randomized age", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum Random\n" + "r= " + str(stats.pearsonr(hannum_y_test, han_coh_pred)[0]), loc= "right")
plt.legend()
plt.savefig("han_coh_model")
plt.show()

now for Horvath WB

In [ ]:
wb_data_28k = ["GSE41037", "GSE20067", "GSE20236", "GSE19711"]

In [ ]:
wb_data_450k = ["GSE41169", "GSE42861"]

In [ ]:
X_28k_wb, y_28k_wb = process_data(wb_data_28k)

In [ ]:
X_450k_wb, y_450k_wb = process_data(wb_data_450k)

In [ ]:
X_450k_wb_shar = X_450k_wb[shared_CpGs]
X_28k_wb_shar = X_28k_wb[shared_CpGs]

In [ ]:
X_wb = pd.concat([X_450k_wb_shar, X_28k_wb_shar])
y_wb = y_450k_wb + y_28k_wb

In [ ]:
del X_28k_wb, y_28k_wb, X_450k_wb, y_450k_wb, X_450k_wb_shar, X_28k_wb_shar

In [ ]:
# wb_model = model.ElasticNetCV(alphas= [0.02, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1], l1_ratio= [0.1, 0.3, 0.5, 0.8, 0.95], selection= "random")
wb_model = model.ElasticNet(alpha= 0.02, l1_ratio= 0.1)

In [ ]:
X_wb = scaler.fit_transform(X_wb)

In [ ]:
X_wb_train, X_wb_test, y_wb_train, y_wb_test = train_test_split(X_wb, y_wb, test_size= 0.20)

In [ ]:
wb_model.fit(X_wb_train, y_wb_train)

In [ ]:
alpha_wb = wb_model.alpha
l1_ratio_wb = wb_model.l1_ratio

# alpha_wb = wb_model.alpha
# l1_ratio_wb = wb_model.l1_ratio

# best so far: 0.02, 0.1
print(alpha_wb, l1_ratio_wb)

In [ ]:
age_pred_wb = wb_model.predict(X_wb_test)

plt.scatter(y_wb_test, age_pred_wb, label= "sample", marker= "+")
plt.xlim(0, 100)
plt.ylim(0, 100)
acc_line_wb = [0, 100]
plt.plot(acc_line_wb, acc_line_wb)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(np.array(y_wb_test), age_pred_wb)[0]), loc= "right")
plt.legend()
plt.savefig("horvath_wb_data_age_model", bbox_inches= "tight")
plt.show()

In [ ]:
del X_wb_train, X_wb_test

now let's do this with random middle ages

In [ ]:
y_wb_coh = y_wb.copy()

In [ ]:
rand_age_ind_wb = []
norm_age_ind_wb = []
for i in range(len(y_wb_coh)):
    age = y_wb_coh[i]
    if age >= 24 and age < 70:
        y_wb_coh[i] = np.random.randint(24, 70)
        rand_age_ind_wb.append(True)
        norm_age_ind_wb.append(False)
    else:
        rand_age_ind_wb.append(False)
        norm_age_ind_wb.append(True)

In [ ]:
wb_X_train, wb_X_test, wb_y_train, wb_y_test, rand_age_ind_train_wb, rand_age_ind_test_wb, norm_age_ind_train_wb, \
norm_age_ind_test_wb = train_test_split(X_wb, y_wb, rand_age_ind_wb, norm_age_ind_wb, test_size= 0.20)

In [ ]:
wb_data_model_coh = model.ElasticNet(alpha= 0.02, l1_ratio= 0.1)

In [ ]:
wb_data_model_coh.fit(wb_X_train, wb_y_train)

In [ ]:
wb_coh_pred = wb_data_model_coh.predict(wb_X_test)

In [ ]:
plt.scatter(np.array(wb_y_test)[norm_age_ind_test_wb], np.array(wb_coh_pred)[norm_age_ind_test_wb], label= "normal age", marker= "+")
plt.scatter(np.array(wb_y_test)[rand_age_ind_test_wb], np.array(wb_coh_pred)[rand_age_ind_test_wb], label= "randomized age", marker= "+")
plt.xlim(0, 100)
plt.ylim(0, 100)
acc_line = [0, 100]
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB Random\n" + "r= " + str(stats.pearsonr(wb_y_test, wb_coh_pred)[0]), loc= "right")
plt.legend()
plt.savefig("wb_coh_model")
plt.show()

### validating Hannum-based and Horvath WB models using external datasets

In [ ]:
brca1_data = ["GSE57285"]

In [ ]:
X_brca1, y_brca1 = process_data(brca1_data)

In [ ]:
_, brca_stat = process_data(brca1_data, info= "brca1")

In [ ]:
X_brca1.head(4)

In [ ]:
X_brca1 = X_brca1[shared_CpGs]

In [ ]:
X_brca1 = scaler.fit_transform(X_brca1, y_brca1)

In [ ]:
wb_y_brca1 = wb_model.predict(X_brca1)

In [ ]:
brca_c0_ind = [0 == code for code in brca_stat]
brca_c1_ind = [1 == code for code in brca_stat]
brca_c2_ind = [2 == code for code in brca_stat]

In [ ]:
# brca_y_v_arr = np.array(brca_y_v)
y_brca1_arr = np.array(y_brca1)

In [ ]:
plt.scatter(y_brca1_arr[brca_c0_ind], wb_y_brca1[brca_c0_ind], color= "blue", label= "healthy brca1 wt", marker= "+")
plt.scatter(y_brca1_arr[brca_c1_ind], wb_y_brca1[brca_c1_ind], color= "red", label= "healthy brca1 mt", marker= "+")
plt.scatter(y_brca1_arr[brca_c2_ind], wb_y_brca1[brca_c2_ind], color= "green", label= "breast cancer brca1 mt", marker= "+")
plt.xlim(0, 100)
plt.ylim(0, 100)
acc_line_brca1 = [0, 100]
plt.plot(acc_line_brca1, acc_line_brca1)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_brca1_arr, wb_y_brca1)[0]), loc= "right")
plt.legend()
plt.savefig("brca1_age_wb")
plt.show()

In [ ]:
plt.scatter(y_brca1_arr[brca_c0_ind], wb_y_brca1[brca_c0_ind], color= "blue", label= "healthy brca1 wt", marker= "+")
plt.xlim(0, 100)
plt.ylim(0, 100)
acc_line_v = [0, 100]
plt.plot(acc_line_v, acc_line_v)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_brca1_arr[brca_c0_ind], wb_y_brca1[brca_c0_ind])[0]), loc= "right")
plt.legend()
plt.savefig("brca1_age_wb_h_brca_wt")
plt.show()

In [ ]:
plt.scatter(y_brca1_arr[brca_c1_ind], wb_y_brca1[brca_c1_ind], color= "red", label= "healthy brca1 mt", marker= "+")
plt.plot(acc_line_brca1, acc_line_brca1)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_brca1_arr[brca_c1_ind], wb_y_brca1[brca_c1_ind])[0]), loc= "right")
plt.legend()
plt.savefig("brca1_age_wb_h_brca1_mt")
plt.show()

In [ ]:
plt.scatter(y_brca1_arr[brca_c2_ind], wb_y_brca1[brca_c2_ind], color= "green", label= "breast cancer brca1 mt", marker= "+")
plt.plot(acc_line_brca1, acc_line_brca1)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_brca1_arr[brca_c2_ind], wb_y_brca1[brca_c2_ind])[0]), loc= "right")
plt.legend()
plt.savefig("brca1_age_wb_canc_brca1_mt")
plt.show()

In [ ]:
# X_brca1 is a 27k dataset so we can't test with Hannum 

let's look at an HIV dataset

In [ ]:
HIV_male_data = ["GSE53840"]

In [ ]:
X_HIV, y_HIV = process_data(HIV_male_data)

In [ ]:
X_HIV["age"] = y_HIV

In [ ]:
# we need to filter for missing ages (they default to 0)
X_HIV = X_HIV[X_HIV["age"] != 0.0]

row_keep = [a != 0.0 for a in y_HIV]
y_HIV = np.array(y_HIV)[row_keep]

In [ ]:
X_HIV.drop(labels= "age", axis= "columns", inplace= True)

In [ ]:
X_HIV.head(4)

In [ ]:
X_HIV_wb = X_HIV[shared_CpGs]

In [ ]:
X_HIV_wb = scaler.fit_transform(X_HIV_wb)

In [ ]:
y_HIV_pred = wb_model.predict(X_HIV_wb)

In [ ]:
y_HIV_arr = np.array(y_HIV)

In [ ]:
plt.scatter(y_HIV_arr, y_HIV_pred, label= "HIV+ male", marker= "+")
acc_line_v = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line_v, acc_line_v)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_HIV_arr, y_HIV_pred)[0]), loc= "right")
plt.legend()
plt.savefig("age_hiv_wb", bbox_inches= "tight")
plt.show()

now let's see how the Hannum based model behaves

In [ ]:
X_HIV_hann, y_HIV_hann = process_data(HIV_male_data)

In [ ]:
X_HIV_hann = X_HIV_hann[hannum_CpGs]

In [ ]:
X_HIV_hann["age"] = y_HIV_hann

In [ ]:
# we need to filter for missing ages (they default to 0)
X_HIV_hann = X_HIV_hann[X_HIV_hann["age"] != 0.0]

row_keep = [a != 0.0 for a in y_HIV_hann]
y_HIV_hann = np.array(y_HIV_hann)[row_keep]

In [ ]:
X_HIV_hann.drop(labels= "age", axis= "columns", inplace= True)

In [ ]:
X_HIV_hann.head(3)

In [ ]:
X_HIV_hann = scaler.fit_transform(X_HIV_hann, y_HIV_hann)

In [ ]:
y_pred_HIV_hann = hannum_data_model.predict(X_HIV_hann)

In [ ]:
plt.scatter(y_HIV_arr, y_pred_HIV_hann, label= "HIV+ male", marker= "+")
plt.plot(acc_line_v, acc_line_v)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum\n" + "r= " + str(stats.pearsonr(y_HIV_arr, y_pred_HIV_hann)[0]), loc= "right")
plt.legend()
plt.savefig("age_hiv_hannum", bbox_inches= "tight")
plt.show()

let's see if a trained OLS regression model will give us good results

In [ ]:
HIV_OLS = model.LinearRegression()

In [ ]:
X_HIV = scaler.fit_transform(X_HIV)

In [ ]:
X_HIV_train, X_HIV_test, y_HIV_train, y_HIV_test = train_test_split(X_HIV, y_HIV_arr, test_size= 20)

In [ ]:
HIV_OLS.fit(X_HIV_train, y_HIV_train)

In [ ]:
y_HIV_OLS_pred = HIV_OLS.predict(X_HIV_test)

In [ ]:
plt.scatter(y_HIV_test, y_HIV_OLS_pred, label= "HIV+ male", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
HIV_OLS_r = stats.pearsonr(y_HIV_test, y_HIV_OLS_pred)[0]
plt.title("DNAm pred. vs actual age, OLS model\n" + "r= " + str(HIV_OLS_r), loc= "right")
plt.legend()
plt.savefig("OLS_HIV")
plt.show()

let's look at a rheumatoid arthritis dataset

In [ ]:
rheu_data = ["GSE42861"]

In [ ]:
X_rheu, y_rheu = process_data(rheu_data)

In [ ]:
_, rheu_stat = process_data(rheu_data, info= "arthritis")

In [ ]:
X_rheu_wb = X_rheu[shared_CpGs]

In [ ]:
X_rheu_wb.head(4)

In [ ]:
X_rheu_wb_arr = scaler.fit_transform(X_rheu_wb)

In [ ]:
wb_rheu_y_pred = wb_model.predict(X_rheu_wb_arr)

In [ ]:
arth_c0_ind = ["normal" == code for code in rheu_stat]
arth_c1_ind = ["rheumatoid arthritis" == code for code in rheu_stat]

In [ ]:
# arth_status_v_arr = np.array(arth_status_v)
y_rheu_arr = np.array(y_rheu)

In [ ]:
plt.scatter(y_rheu_arr[arth_c0_ind], wb_rheu_y_pred[arth_c0_ind], color= "blue", label= "normal", marker= "+")
plt.scatter(y_rheu_arr[arth_c1_ind], wb_rheu_y_pred[arth_c1_ind], color= "red", label= "rheumatoid arthritis", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_rheu_arr, wb_rheu_y_pred)[0]), loc= "right")
plt.legend()
plt.savefig("wb_arthritis")
plt.show()

In [ ]:
wb_coh_pred_arth = wb_data_model_coh.predict(X_rheu_wb_arr)

In [ ]:
plt.scatter(y_rheu_arr[arth_c0_ind], wb_coh_pred_arth[arth_c0_ind], color= "blue", label= "normal", marker= "+")
plt.scatter(y_rheu_arr[arth_c1_ind], wb_coh_pred_arth[arth_c1_ind], color= "red", label= "rheumatoid arthritis", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB Random\n" + "r= " + str(stats.pearsonr(y_rheu_arr, wb_coh_pred_arth)[0]), loc= "right")
plt.legend()
plt.savefig("wb_coh_arthritis")
plt.show()

now let's see how the Hannum based model performs

In [ ]:
X_rheu_hann = X_rheu[hannum_CpGs]

In [ ]:
X_rheu_hann = scaler.fit_transform(X_rheu_hann, y_rheu)

In [ ]:
hann_rheu_y_pred = hannum_data_model.predict(X_rheu_hann)

In [ ]:
plt.scatter(y_rheu_arr[arth_c0_ind], hann_rheu_y_pred[arth_c0_ind], color= "blue", label= "normal", marker= "+")
plt.scatter(y_rheu_arr[arth_c1_ind], hann_rheu_y_pred[arth_c1_ind], color= "red", label= "rheumatoid arthritis", marker= "+")
acc_line = [0, 100]
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum\n" + "r= " + str(stats.pearsonr(y_rheu_arr, hann_rheu_y_pred)[0]), loc= "right")
plt.legend()
plt.savefig("hannum_arthritis")
plt.show()

In [ ]:
han_coh_pred_arth = hannum_data_model_coh.predict(X_rheu_hann)

In [ ]:
plt.scatter(y_rheu_arr[arth_c0_ind], han_coh_pred_arth[arth_c0_ind], color= "blue", label= "normal", marker= "+")
plt.scatter(y_rheu_arr[arth_c1_ind], han_coh_pred_arth[arth_c1_ind], color= "red", label= "rheumatoid arthritis", marker= "+")
acc_line = [0, 100]
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum Random\n" + "r= " + str(stats.pearsonr(y_rheu_arr, han_coh_pred_arth)[0]), loc= "right")
plt.legend()
plt.savefig("han_arthritis_coh")
plt.show()

let's see if a trained OLS regression model will give us good results

In [ ]:
rheu_OLS = model.LinearRegression()

In [ ]:
X_rheu_arr = scaler.fit_transform(X_rheu)

In [ ]:
X_rheu_arr_train, X_rheu_arr_test, y_rheu_train, y_rheu_test, c0_ind_train, c0_ind_test, c1_ind_train, c1_ind_test = train_test_split(X_rheu_arr, y_rheu, arth_c0_ind, arth_c1_ind)

In [ ]:
rheu_OLS.fit(X_rheu_arr_train, y_rheu_train)

In [ ]:
y_rheu_OLS_pred = rheu_OLS.predict(X_rheu_arr_test)

In [ ]:
y_rheu_test_arr = np.array(y_rheu_test)

In [ ]:
plt.scatter(y_rheu_test_arr[c0_ind_test], y_rheu_OLS_pred[c0_ind_test], color= "blue", label= "normal", marker= "+")
plt.scatter(y_rheu_test_arr[c1_ind_test], y_rheu_OLS_pred[c1_ind_test], color= "red", label= "rheumatoid arthritis", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, OLS model\n" + "r= " + str(stats.pearsonr(y_rheu_test_arr, y_rheu_OLS_pred)[0]), loc= "right")
plt.legend()
plt.savefig("OLS_arthritis")
plt.show()

In [ ]:
plt.scatter(y_rheu_test_arr[c0_ind_test], y_rheu_OLS_pred[c0_ind_test], color= "blue", label= "normal", marker= "+")
# plt.scatter(y_rheu_test_arr[c1_ind_test], y_rheu_OLS_pred[c1_ind_test], color= "red", label= "rheumatoid arthritis", marker= "+")
acc_line = [0, 100]
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, OLS model\n" + "r= " + str(stats.pearsonr(y_rheu_test_arr[c0_ind_test], y_rheu_OLS_pred[c0_ind_test])[0]), loc= "right")
plt.legend()
plt.savefig("OLS_arthritis_normal")
plt.show()

In [ ]:
# plt.scatter(y_rheu_test_arr[c0_ind_test], y_rheu_OLS_pred[c0_ind_test], color= "blue", label= "normal", marker= "+")
plt.scatter(y_rheu_test_arr[c1_ind_test], y_rheu_OLS_pred[c1_ind_test], color= "red", label= "rheumatoid arthritis", marker= "+")
acc_line = [0, 100]
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, OLS model\n" + "r= " + str(stats.pearsonr(y_rheu_test_arr[c1_ind_test], y_rheu_OLS_pred[c1_ind_test])[0]), loc= "right")
plt.legend()
plt.savefig("OLS_arthritis_arthritis")
plt.show()

let's look at a Crohn's disease dataset

In [ ]:
crohns_data = ["GSE32148"]

In [ ]:
X_crohns, y_crohns = process_data(crohns_data)

In [ ]:
# some samples are missing
X_crohns.shape

In [ ]:
y_crohns[0]

In [ ]:
_, crohns_stat = process_data(crohns_data, info= "crohns")

In [ ]:
crohns_c0_ind = ["Crohn's" == code for code in crohns_stat]
crohns_c1_ind = ["normal" == code for code in crohns_stat]
crohns_c2_ind = ["ulcerative" == code for code in crohns_stat]

In [ ]:
y_crohns_arr = np.array(y_crohns)

In [ ]:
X_crohns_wb = X_crohns[shared_CpGs]

In [ ]:
X_crohns_wb = scaler.fit_transform(X_crohns_wb)

In [ ]:
wb_crohns_y_pred = wb_model.predict(X_crohns_wb)

In [ ]:
plt.scatter(y_crohns_arr[crohns_c0_ind], wb_crohns_y_pred[crohns_c0_ind], color= "blue", label= "crohn's", marker= "+")
plt.scatter(y_crohns_arr[crohns_c1_ind], wb_crohns_y_pred[crohns_c1_ind], color= "red", label= "normal", marker= "+")
plt.scatter(y_crohns_arr[crohns_c2_ind], wb_crohns_y_pred[crohns_c2_ind], color= "green", label= "ulcerative colitis", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_crohns_arr, wb_crohns_y_pred)[0]), loc= "right")
plt.legend()
plt.savefig("wb_crohns")
plt.show()

In [ ]:
plt.scatter(y_crohns_arr[crohns_c0_ind], wb_crohns_y_pred[crohns_c0_ind], color= "blue", label= "crohn's", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_crohns_arr[crohns_c0_ind], wb_crohns_y_pred[crohns_c0_ind])[0]), loc= "right")
plt.legend()
plt.savefig("wb_crohns_crohns")
plt.show()

In [ ]:
plt.scatter(y_crohns_arr[crohns_c1_ind], wb_crohns_y_pred[crohns_c1_ind], color= "red", label= "normal", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_crohns_arr[crohns_c1_ind], wb_crohns_y_pred[crohns_c1_ind])[0]), loc= "right")
plt.legend()
plt.savefig("wb_crohns_normal")
plt.show()

In [ ]:
plt.scatter(y_crohns_arr[crohns_c2_ind], wb_crohns_y_pred[crohns_c2_ind], color= "green", label= "ulcerative colitis", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_crohns_arr[crohns_c2_ind], wb_crohns_y_pred[crohns_c2_ind])[0]), loc= "right")
plt.legend()
plt.savefig("wb_crohns_ulcer_col")
plt.show()

let's see how the Hannum based clock performs

In [ ]:
X_crohns_hann = X_crohns[hannum_CpGs]

In [ ]:
X_crohns_hann = scaler.fit_transform(X_crohns_hann, y_crohns)

In [ ]:
hann_crohns_y_pred = hannum_data_model.predict(X_crohns_hann)

In [ ]:
plt.scatter(y_crohns_arr[crohns_c0_ind], hann_crohns_y_pred[crohns_c0_ind], color= "blue", label= "crohn's", marker= "+")
plt.scatter(y_crohns_arr[crohns_c1_ind], hann_crohns_y_pred[crohns_c1_ind], color= "red", label= "normal", marker= "+")
plt.scatter(y_crohns_arr[crohns_c2_ind], hann_crohns_y_pred[crohns_c2_ind], color= "green", label= "ulcerative colitis", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum\n" + "r= " + str(stats.pearsonr(y_crohns_arr, hann_crohns_y_pred)[0]), loc= "right")
plt.legend()
plt.savefig("hannum_crohns")
plt.show()

In [ ]:
plt.scatter(y_crohns_arr[crohns_c0_ind], hann_crohns_y_pred[crohns_c0_ind], color= "blue", label= "crohn's", marker= "+")
# plt.scatter(y_crohns_arr[crohns_c1_ind], hann_crohns_y_pred[crohns_c1_ind], color= "red", label= "normal", marker= "+")
# plt.scatter(y_crohns_arr[crohns_c2_ind], hann_crohns_y_pred[crohns_c2_ind], color= "green", label= "ulcerative colitis", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum\n" + "r= " + str(stats.pearsonr(y_crohns_arr[crohns_c0_ind], hann_crohns_y_pred[crohns_c0_ind])[0]), loc= "right")
plt.legend()
plt.savefig("hannum_crohns_crohns")
plt.show()

In [ ]:
plt.scatter(y_crohns_arr[crohns_c1_ind], hann_crohns_y_pred[crohns_c1_ind], color= "red", label= "normal", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum\n" + "r= " + str(stats.pearsonr(y_crohns_arr[crohns_c1_ind], hann_crohns_y_pred[crohns_c1_ind])[0]), loc= "right")
plt.legend()
plt.savefig("hannum_crohns_normal")
plt.show()

In [ ]:
plt.scatter(y_crohns_arr[crohns_c2_ind], hann_crohns_y_pred[crohns_c2_ind], color= "green", label= "ulcerative colitis", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum\n" + "r= " + str(stats.pearsonr(y_crohns_arr[crohns_c2_ind], hann_crohns_y_pred[crohns_c2_ind])[0]), loc= "right")
plt.legend()
plt.savefig("hannum_crohns_ulcer_col")
plt.show()

let's look at a down syndrome dataset

In [ ]:
down_syn_data = ["GSE52588"]

In [ ]:
X_ds, y_ds = process_data(down_syn_data)

In [ ]:
y_ds = np.array(y_ds)

In [ ]:
_, ds_status = process_data(down_syn_data, info= "down syndrome")

In [ ]:
ds_c0_ind = ["Down" == code for code in ds_status]
ds_c1_ind = ["healthy" == code for code in ds_status]

In [ ]:
X_ds_han = X_ds[hannum_CpGs]

In [ ]:
X_ds_han = scaler.fit_transform(X_ds_han)

In [ ]:
y_ds_han = hannum_data_model.predict(X_ds_han)

In [ ]:
plt.scatter(y_ds[ds_c1_ind], y_ds_han[ds_c1_ind], color= "blue", label= "normal", marker= "+")
plt.scatter(y_ds[ds_c0_ind], y_ds_han[ds_c0_ind], color= "red", label= "down syndrome", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum\n" + "r= " + str(stats.pearsonr(y_ds, y_ds_han)[0]), loc= "right")
plt.legend()
plt.savefig("han_ds")
plt.show()

In [ ]:
y_ds_han_coh = hannum_data_model_coh.predict(X_ds_han)

In [ ]:
plt.scatter(y_ds[ds_c1_ind], y_ds_han[ds_c1_ind], color= "blue", label= "normal", marker= "+")
plt.scatter(y_ds[ds_c0_ind], y_ds_han[ds_c0_ind], color= "red", label= "down syndrome", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum Random\n" + "r= " + str(stats.pearsonr(y_ds, y_ds_han_coh)[0]), loc= "right")
plt.legend()
plt.savefig("han_ds_coh")
plt.show()

In [ ]:
X_ds_wb = X_ds[shared_CpGs]

In [ ]:
X_ds_wb = scaler.fit_transform(X_ds_wb)

In [ ]:
wb_ds_y_pred = wb_model.predict(X_ds_wb)

In [ ]:
plt.scatter(y_ds[ds_c1_ind], wb_ds_y_pred[ds_c1_ind], color= "blue", label= "normal", marker= "+")
plt.scatter(y_ds[ds_c0_ind], wb_ds_y_pred[ds_c0_ind], color= "red", label= "down syndrome", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB\n" + "r= " + str(stats.pearsonr(y_ds, wb_ds_y_pred)[0]), loc= "right")
plt.legend()
plt.savefig("wb_ds")
plt.show()

In [ ]:
wb_ds_y_pred_coh = wb_data_model_coh.predict(X_ds_wb)

In [ ]:
plt.scatter(y_ds[ds_c1_ind], wb_ds_y_pred_coh[ds_c1_ind], color= "blue", label= "normal", marker= "+")
plt.scatter(y_ds[ds_c0_ind], wb_ds_y_pred_coh[ds_c0_ind], color= "red", label= "down syndrome", marker= "+")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB Random\n" + "r= " + str(stats.pearsonr(y_ds, wb_ds_y_pred_coh)[0]), loc= "right")
plt.legend()
plt.savefig("wb_ds_coh")
plt.show()

let's see if a trained OLS regression model will give us good results

In [ ]:
ds_OLS = model.LinearRegression()

In [ ]:
X_ds = scaler.fit_transform(X_ds)

In [ ]:
X_ds_train, X_ds_test, y_ds_train, y_ds_test, ds_c0_ind_train, ds_c0_ind_test, ds_c1_ind_train, ds_c1_ind_test = \
train_test_split(X_ds, y_ds, ds_c0_ind, ds_c1_ind, test_size= 20)

In [ ]:
ds_OLS.fit(X_ds_train, y_ds_train)

In [ ]:
y_ds_OLS_pred = ds_OLS.predict(X_ds_test)

In [ ]:
plt.scatter(y_ds_test[ds_c1_ind_test], y_ds_OLS_pred[ds_c1_ind_test], label= "normal", marker= "+", color= "blue")
plt.scatter(y_ds_test[ds_c0_ind_test], y_ds_OLS_pred[ds_c0_ind_test], label= "down syndrome", marker= "+", color= "red")
acc_line = [0, 100]
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.plot(acc_line, acc_line)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
ds_OLS_r = stats.pearsonr(y_ds_test, y_ds_OLS_pred)[0]
plt.title("DNAm pred. vs actual age, OLS model\n" + "r= " + str(ds_OLS_r), loc= "right")
plt.legend()
plt.savefig("ds_OLS")
plt.show()

let's test random models on HIV+ male set

In [ ]:
y_pred_HIV_han_coh = hannum_data_model_coh.predict(X_HIV_hann)

In [ ]:
plt.scatter(y_HIV_arr, y_pred_HIV_han_coh, label= "HIV+ male", marker= "+")
plt.plot(acc_line_v, acc_line_v)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Hannum Random\n" + "r= " + str(stats.pearsonr(y_HIV_arr, y_pred_HIV_han_coh)[0]), loc= "right")
plt.legend()
plt.savefig("hiv_han_coh", bbox_inches= "tight")
plt.show()

In [ ]:
y_pred_HIV_wb_coh = wb_data_model_coh.predict(X_HIV)

In [ ]:
plt.scatter(y_HIV_arr, y_pred_HIV_wb_coh, label= "HIV+ male", marker= "+")
plt.plot(acc_line_v, acc_line_v)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB Random\n" + "r= " + str(stats.pearsonr(y_HIV_arr, y_pred_HIV_wb_coh)[0]), loc= "right")
plt.legend()
plt.savefig("wb_coh_hiv", bbox_inches= "tight")
plt.show()

let's test random model on BRCA1 dataset

In [ ]:
wb_y_brca1_coh = wb_data_model_coh.predict(X_brca1)

In [ ]:
plt.scatter(y_brca1_arr[brca_c0_ind], wb_y_brca1_coh[brca_c0_ind], color= "blue", label= "healthy brca1 wt", marker= "+")
plt.scatter(y_brca1_arr[brca_c1_ind], wb_y_brca1_coh[brca_c1_ind], color= "red", label= "healthy brca1 mt", marker= "+")
plt.scatter(y_brca1_arr[brca_c2_ind], wb_y_brca1_coh[brca_c2_ind], color= "green", label= "breast cancer brca1 mt", marker= "+")
plt.xlim(0, 100)
plt.ylim(0, 100)
acc_line_brca1 = [0, 100]
plt.plot(acc_line_brca1, acc_line_brca1)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB Random\n" + "r= " + str(stats.pearsonr(y_brca1_arr, wb_y_brca1_coh)[0]), loc= "right")
plt.legend()
plt.savefig("brca1_age_wb_coh")
plt.show()

In [ ]:
plt.scatter(y_brca1_arr[brca_c0_ind], wb_y_brca1_coh[brca_c0_ind], color= "blue", label= "healthy brca1 wt", marker= "+")
plt.xlim(0, 100)
plt.ylim(0, 100)
acc_line_v = [0, 100]
plt.plot(acc_line_v, acc_line_v)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB Random\n" + "r= " + str(stats.pearsonr(y_brca1_arr[brca_c0_ind], wb_y_brca1_coh[brca_c0_ind])[0]), loc= "right")
plt.legend()
plt.savefig("brca1_age_wb_h_brca_wt_coh")
plt.show()

In [ ]:
plt.scatter(y_brca1_arr[brca_c1_ind], wb_y_brca1_coh[brca_c1_ind], color= "red", label= "healthy brca1 mt", marker= "+")
plt.plot(acc_line_brca1, acc_line_brca1)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB Random\n" + "r= " + str(stats.pearsonr(y_brca1_arr[brca_c1_ind], wb_y_brca1_coh[brca_c1_ind])[0]), loc= "right")
plt.legend()
plt.savefig("brca1_age_wb_h_brca1_mt_coh")
plt.show()

In [ ]:
plt.scatter(y_brca1_arr[brca_c2_ind], wb_y_brca1_coh[brca_c2_ind], color= "green", label= "breast cancer brca1 mt", marker= "+")
plt.plot(acc_line_brca1, acc_line_brca1)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.xlabel("actual age")
plt.ylabel("DNAm pred. age")
plt.title("DNAm pred. vs actual age, Horvath WB Random\n" + "r= " + str(stats.pearsonr(y_brca1_arr[brca_c2_ind], wb_y_brca1_coh[brca_c2_ind])[0]), loc= "right")
plt.legend()
plt.savefig("brca1_age_wb_canc_brca1_mt_coh")
plt.show()

using the human methylome to predict stock market values during the crash

In [ ]:
stock_data = pd.read_csv("dow-jones-1929-bear-market.csv")
stock_data.head(5)

In [ ]:
# converting date strings to date objects
for i in range(stock_data["date"].shape[0]):
    date_match = re.match(r"(\d+)/(\d+)/(\d+)", stock_data["date"].iloc[i])
    year = int(date_match.group(3))
    month = int(date_match.group(1))
    day = int(date_match.group(2))
    date_obj = date(year, month, day)
    stock_data["date"].iloc[i] = date_obj

In [ ]:
stock_data.head(3)

In [ ]:
dates = np.array(stock_data["date"])
values = np.array(stock_data[" value"])

In [ ]:
plt.plot(dates, values)
plt.xlabel("date")
plt.ylabel("value")
plt.xticks([date(1929, 9, 3), date(1930, 9, 3), date(1931, 9, 3), date(1932, 7, 3)])
plt.title("stock market value")
plt.savefig("stock_market_value")
plt.show()

In [ ]:
# map ages to indices in dates
min_age_han = min(hannum_y)
max_age_han = max(hannum_y)
age_diff_han = max_age_han - min_age_han

# we want min_age_han to map to 0 and max_age_han
# to map to len(dates) - 1
step_han = (len(dates) - 1) / age_diff_han

date_ind = []    # in the same order as hannum_y
for age in hannum_y:
    ind = int((age - min_age_han) * step_han)
    date_ind.append(ind)

date_ind = np.array(date_ind)

In [ ]:
age_values = []    # in same order as hannum_y
for ind in date_ind:
    age_values.append(values[ind])

In [ ]:
han_X_train_stock, han_X_test_stock, han_y_train_stock, han_y_test_stock = train_test_split(hannum_X,
                                                                                            age_values,
                                                                                            test_size= 0.20)

In [ ]:
han_stock_model = model.ElasticNet(alpha= 0.1, l1_ratio= 0.95)

In [ ]:
han_stock_model.fit(han_X_train_stock, han_y_train_stock)

In [ ]:
han_stock_pred = han_stock_model.predict(han_X_test_stock)

In [ ]:
plt.scatter(han_y_test_stock, han_stock_pred, marker= "+", label= "stock value")
acc_line = [0, 400]
plt.plot(acc_line, acc_line)
plt.xlim(0, 400)
plt.ylim(0, 400)
plt.xlabel("actual stock value")
plt.ylabel("DNAm pred. stock value")
r_han_stock = stats.pearsonr(han_y_test_stock, han_stock_pred)[0]
plt.title("DNAm pred. vs actual stock value, Hannum Stocks\n" + "r= " + str(r_han_stock), loc= "right")
plt.legend()
plt.savefig("han_stock")
plt.show()

let's train a model using random stock values in the middle

In [ ]:
han_y_stock_rand = np.array(age_values.copy())

In [ ]:
rand_stock_ind_han = []
norm_stock_ind_han = []
for i in range(len(han_y_stock_rand)):
    stock_val = han_y_stock_rand[i]
    if stock_val >= 100 and stock_val < 250:
        han_y_stock_rand[i] = np.random.randint(90, 260)
        rand_stock_ind_han.append(True)
        norm_stock_ind_han.append(False)
    else:
        rand_stock_ind_han.append(False)
        norm_stock_ind_han.append(True)

In [ ]:
han_X_train_stock_rand, han_X_test_stock_rand, han_y_train_stock_rand, han_y_test_stock_rand, \
rand_stock_ind_han_train, rand_stock_ind_han_test, norm_stock_ind_han_train, norm_stock_ind_han_test = \
train_test_split(hannum_X, age_values, rand_stock_ind_han, norm_stock_ind_han, test_size= 0.20)

In [ ]:
rand_stock_han_model = model.ElasticNet(alpha= 0.1, l1_ratio= 0.95)

In [ ]:
rand_stock_han_model.fit(han_X_train_stock_rand, han_y_train_stock_rand)

In [ ]:
rand_stock_pred_han = np.array(rand_stock_han_model.predict(han_X_test_stock_rand))

In [ ]:
plt.scatter(np.array(han_y_test_stock_rand)[norm_stock_ind_han_test], rand_stock_pred_han[norm_stock_ind_han_test], label= "normal stock value", marker= "+")
plt.scatter(np.array(han_y_test_stock_rand)[rand_stock_ind_han_test], rand_stock_pred_han[rand_stock_ind_han_test], label= "random stock value", marker= "+")
plt.xlim(0, 400)
plt.ylim(0, 400)
acc_line = [0, 400]
plt.plot(acc_line, acc_line)
plt.xlabel("actual stock value")
plt.ylabel("DNAm pred. stock value")
han_stock_rand_r = stats.pearsonr(han_y_test_stock_rand, rand_stock_pred_han)[0]
plt.title("DNAm pred. vs actual stock value, Hannum Stocks Random\n" + "r= " + str(han_stock_rand_r), loc= "right")
plt.legend()
plt.savefig("han_stocks_rand")
plt.show()

now let's do with horvath

In [ ]:
# map ages to indices in dates
min_age_wb = min(y_wb)
max_age_wb = max(y_wb)
age_diff_wb = max_age_wb - min_age_wb

# we want min_age_wb to map to 0 and max_age_wb
# to map to len(dates) - 1
step_wb = (len(dates) - 1) / age_diff_wb

date_ind_wb = []    # in the same order as hannum_y
for age in y_wb:
    ind = int((age - min_age_wb) * step_wb)
    date_ind_wb.append(ind)

date_ind_wb = np.array(date_ind_wb)

In [ ]:
age_values_wb = []    # in same order as hannum_y
for ind in date_ind_wb:
    age_values_wb.append(values[ind])

In [ ]:
wb_X_train_stock, wb_X_test_stock, wb_y_train_stock, wb_y_test_stock = train_test_split(X_wb,
                                                                                        age_values_wb,
                                                                                        test_size= 0.20)

In [ ]:
wb_stock_model = model.ElasticNet(alpha= 0.02, l1_ratio= 0.1)

In [ ]:
wb_stock_model.fit(wb_X_train_stock, wb_y_train_stock)

In [ ]:
wb_stock_pred = wb_stock_model.predict(wb_X_test_stock)

In [ ]:
plt.scatter(wb_y_test_stock, wb_stock_pred, marker= "+", label= "stock value")
acc_line = [0, 400]
plt.plot(acc_line, acc_line)
plt.xlim(0, 400)
plt.ylim(0, 400)
plt.xlabel("actual stock value")
plt.ylabel("DNAm pred. stock value")
r_wb_stock = stats.pearsonr(wb_y_test_stock, wb_stock_pred)[0]
plt.title("DNAm pred. vs actual stock value, Horvath WB Stocks\n" + "r= " + str(r_wb_stock), loc= "right")
plt.legend()
plt.savefig("wb_stock")
plt.show()

let's train a model using random stock values in the middle

In [ ]:
wb_y_stock_rand = np.array(age_values_wb.copy())

In [ ]:
rand_stock_ind_wb = []
norm_stock_ind_wb = []
for i in range(len(wb_y_stock_rand)):
    stock_val = wb_y_stock_rand[i]
    if stock_val >= 95 and stock_val < 275:
        wb_y_stock_rand[i] = np.random.randint(95, 275)
        rand_stock_ind_wb.append(True)
        norm_stock_ind_wb.append(False)
    else:
        rand_stock_ind_wb.append(False)
        norm_stock_ind_wb.append(True)

In [ ]:
wb_X_train_stock_rand, wb_X_test_stock_rand, wb_y_train_stock_rand, wb_y_test_stock_rand, \
rand_stock_ind_wb_train, rand_stock_ind_wb_test, norm_stock_ind_wb_train, norm_stock_ind_wb_test = \
train_test_split(X_wb, age_values_wb, rand_stock_ind_wb, norm_stock_ind_wb, test_size= 0.20)

In [ ]:
rand_stock_wb_model = model.ElasticNet(alpha= 0.02, l1_ratio= 0.1)

In [ ]:
rand_stock_wb_model.fit(wb_X_train_stock_rand, wb_y_train_stock_rand)

In [ ]:
rand_stock_pred_wb = np.array(rand_stock_wb_model.predict(wb_X_test_stock_rand))

In [ ]:
plt.scatter(np.array(wb_y_test_stock_rand)[norm_stock_ind_wb_test], rand_stock_pred_wb[norm_stock_ind_wb_test], label= "normal stock value", marker= "+")
plt.scatter(np.array(wb_y_test_stock_rand)[rand_stock_ind_wb_test], rand_stock_pred_wb[rand_stock_ind_wb_test], label= "random stock value", marker= "+")
plt.xlim(0, 400)
plt.ylim(0, 400)
acc_line = [0, 400]
plt.plot(acc_line, acc_line)
plt.xlabel("actual stock value")
plt.ylabel("DNAm pred. stock value")
wb_stock_rand_r = stats.pearsonr(wb_y_test_stock_rand, rand_stock_pred_wb)[0]
plt.title("DNAm pred. vs actual stock value, Horvath WB Stocks Random\n" + "r= " + str(wb_stock_rand_r), loc= "right")
plt.legend()
plt.savefig("wb_stocks_rand")
plt.show()